# Setup

Import Semantic Kernel SDK from pypi.org

In [1]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


'1.28.1'

Initial configuration for the notebook to run properly.

In [6]:
# Make sure paths are correct for the imports

import os
import sys
from pprint import pprint
notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

Let's define our kernel for this example.

In [2]:
from semantic_kernel import Kernel

kernel = Kernel()

We will load our settings and get the LLM service to use for the notebook.

In [3]:
from services import Service

from samples.service_settings import ServiceSettings

service_settings = ServiceSettings(env_file_path=os.path.join(notebook_dir, ".env"))

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [4]:
print(service_settings)

env_file_path='/Users/mfilipav/dev/semantic-kernel/python/samples/getting_started/.env' env_file_encoding='utf-8' global_llm_service='AzureOpenAI'


We now configure our Chat Completion service on the kernel.

In [8]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )
pprint(kernel)
"""
Kernel(
    retry_mechanism=PassThroughWithoutRetry(),
    services={
        'default': AzureChatCompletion(
            ai_model_id='gpt-4o',
            service_id='default',
            instruction_role='system',
            client=<openai.lib.azure.AsyncAzureOpenAI object at 0x12099b310>,
            ai_model_type=<OpenAIModelTypes.CHAT: 'chat'>,
            prompt_tokens=0,
            completion_tokens=0,
            total_tokens=0
        )
    },
    ai_service_selector=<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x120d0ee90>,
    plugins={},
    function_invocation_filters=[],
    prompt_rendering_filters=[],
    auto_function_invocation_filters=[]
)
"""

Kernel(retry_mechanism=PassThroughWithoutRetry(), services={'default': AzureChatCompletion(ai_model_id='gpt-4o', service_id='default', instruction_role='system', client=<openai.lib.azure.AsyncAzureOpenAI object at 0x120c12010>, ai_model_type=<OpenAIModelTypes.CHAT: 'chat'>, prompt_tokens=0, completion_tokens=0, total_tokens=0)}, ai_service_selector=<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x120d0ee90>, plugins={}, function_invocation_filters=[], prompt_rendering_filters=[], auto_function_invocation_filters=[])


"\nKernel(\n    retry_mechanism=PassThroughWithoutRetry(),\n    services={\n        'default': AzureChatCompletion(\n            ai_model_id='gpt-4o',\n            service_id='default',\n            instruction_role='system',\n            client=<openai.lib.azure.AsyncAzureOpenAI object at 0x12099b310>,\n            ai_model_type=<OpenAIModelTypes.CHAT: 'chat'>,\n            prompt_tokens=0,\n            completion_tokens=0,\n            total_tokens=0\n        )\n    },\n    ai_service_selector=<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x120d0ee90>,\n    plugins={},\n    function_invocation_filters=[],\n    prompt_rendering_filters=[],\n    auto_function_invocation_filters=[]\n)\n"

# Run a Semantic Function

**Step 3**: Load a Plugin and run a semantic function:


In [9]:
plugin = kernel.add_plugin(
    parent_directory="../../../prompt_template_samples/", plugin_name="FunPlugin"
)

In [13]:
from semantic_kernel.functions import KernelArguments

joke_function = plugin["Joke"]

joke = await kernel.invoke(
    function=joke_function,
    arguments=KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

Why did the time traveler bring a suitcase full of sandwiches to the dinosaur age?

Because he heard the T. rex was *terrible* at handling fast food!
